# Stage 13 Homework Starter — Productization

## Objective
Deploy your trained model as a **reusable, handoff-ready API or dashboard** and finalize your project for reproducibility and clarity.

## Steps
1. Create a mock, very basic analysis in a notebook.
2. Clean your notebook by removing exploratory cells and documenting your code.
3. Move reusable functions into `/src/`.
4. Load your trained model from Stage 12 or earlier stages.
5. Pickle/save the model and test reload.
6. Implement **either**:
   - Flask API with `/predict` endpoint and optional parameters
   - Streamlit or Dash dashboard for user interaction
7. Include:
   - Error handling for invalid inputs
   - `requirements.txt` for reproducibility
   - Documentation in `README.md`
8. Test your deployment locally and provide evidence.
9. Organize project folders and finalize notebooks for handoff.

## 1. Create mock, very basic analysis

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('../../project/data/processed/VIX_S&P500_features.csv', parse_dates = ['date'])
df.head()

,date,vix_close,vix_high,vix_low,vix_open,sp500_close,sp500_high,sp500_low,sp500_open,sp500_volume,log_sp500_close,vix_delta,vix_spread,sp500_delta,sp500_spread
0,2019-08-28,19.350000,21.639999,19.10,20.549999,2887.939941,2890.030029,2853.050049,2861.280029,3102480000,7.968299,-1.199999,2.539999,26.659912,36.979980
1,2019-08-29,17.879999,19.200001,17.60,19.020000,2924.580078,2930.500000,2905.669922,2910.370117,3177150000,7.980906,-1.140001,1.600000,14.209961,24.830078
2,2019-08-30,18.980000,19.180000,17.09,17.940001,2926.459961,2940.429932,2913.320068,2937.090088,3009910000,7.981549,1.039999,2.090000,-10.630127,27.109863
3,2019-09-03,19.660000,21.150000,19.41,20.959999,2906.270020,2914.389893,2891.850098,2909.010010,3427830000,7.974626,-1.299999,1.740000,-2.739990,22.539795
4,2019-09-04,17.330000,18.830000,17.26,18.230000,2937.780029,2938.840088,2921.860107,2924.669922,3167900000,7.985409,-0.900000,1.570000,13.110107,16.979980


## 2. Notebook Cleanup
Remove exploratory cells and document your code.

In [2]:
print("Notebook cleaned and ready for handoff.")

Notebook cleaned and ready for handoff.


## 3. Move reusable functions to /src/
Create src/utils.py and store functions there.

In [3]:
import sys
import os

sys.path.append(os.path.abspath(''))
from src.utils import *

print('src/utils.py successfully implemented.')

src/utils.py successfully implemented.


## 4. Folder Structure Reminder

Ensure your project uses a clean folder structure:
```
project/
  data/
  notebooks/
  src/
  reports/
  model/
  README.md
```
For API/Dashboard: minimal example:
```
project/
    app.py
    model.pkl
    requirements.txt
    README.md
```

## 5. Pickle / Save Final Model

In [4]:
# Transformed regression with delta & train-test split

import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import scipy.stats as st

X = df[['vix_close']]
y = df['log_sp500_close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True)
lr = LinearRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared = False)
print(f'Baseline R² = {r2:.4f}, RMSE = {rmse:.6f}')

Baseline R² = 0.2504, RMSE = 0.184766


In [5]:
import pickle

os.makedirs('model', exist_ok = True)

with open('model/model.pkl', 'wb') as f:
    pickle.dump(lr, f)

with open('model/model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

print(loaded_model.predict([[12]]))
print(loaded_model.predict([[15]]))
print(loaded_model.predict([[18]]))
print(loaded_model.predict([[21]]))
print(loaded_model.predict([[24]]))
print(loaded_model.predict([[27]]))

[8.47337376]
[8.43476547]
[8.39615718]
[8.35754889]
[8.3189406]
[8.28033231]


/Users/willwu/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/willwu/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/willwu/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/willwu/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/willwu/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/Users/willwu/anaconda3/lib/python3.10/s

## 6. Flask API Starter

### Implement Flask endpoints for /predict and /plot

In [19]:
from pathlib import Path
img_dir = Path('images')
img_dir.mkdir(parents = True, exist_ok = True)

def savefig(name):
    plt.tight_layout()
    plt.savefig(img_dir / name, dpi = 300)
    print(f'Saved {name}')

In [29]:
import jinja2
try:
    from markupsafe import Markup, escape
    jinja2.Markup = Markup
    jinja2.escape = escape
except Exception as e:
    raise ImportError("markupsafe is required for Jinja2 >= 3.1") from e

import os, io, base64, threading, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from flask import Flask, request, jsonify

DATA_PATH = '../../project/data/processed/VIX_S&P500_features.csv'

def _load_df():
    if not os.path.exists(DATA_PATH):
        raise FileNotFoundError(f"Data not found at {DATA_PATH}")
    df = pd.read_csv(DATA_PATH)
    if "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"])
        df = df.sort_values("date")
    return df

def plot_log_sp500_close(df):
    """Return a Matplotlib figure for log_sp500_close vs time."""
    if "log_sp500_close" not in df.columns:
        raise ValueError("Column 'log_sp500_close' not found. Create it before plotting.")
    fig = plt.figure(figsize=(10, 4))
    plt.plot(df["date"], df["log_sp500_close"])
    plt.title("log_sp500_close vs Time")
    plt.xlabel("Date")
    plt.ylabel("log_sp500_close")
    plt.tight_layout()
    savefig('Log S&P 500 over Time.png')
    return fig

def plot_vix_close(df):
    """Return a Matplotlib figure for vix_close vs time."""
    if "vix_close" not in df.columns:
        raise ValueError("Column 'vix_close' not found.")
    fig = plt.figure(figsize=(10, 4))
    plt.plot(df["date"], df["vix_close"])
    plt.title("vix_close vs Time")
    plt.xlabel("Date")
    plt.ylabel("vix_close")
    plt.tight_layout()
    savefig('VIX over Time.png')
    return fig

def _fig_to_img_html(fig, title=None):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    img_b64 = base64.b64encode(buf.read()).decode("utf-8")
    plt.close(fig)
    t = jinja2.escape(title or "")
    return f"<h3>{t}</h3><img src='data:image/png;base64,{img_b64}'/>"

MODEL_PATH = "model/model.pkl"
if os.path.exists(MODEL_PATH):
    with open(MODEL_PATH, "rb") as f:
        model = pickle.load(f)
else:
    class _DummyModel:
        def predict(self, X):
            X = np.asarray(X)
            w = np.array([0.002, -0.0001])[:X.shape[1]]
            b = 0.05
            return (X @ w) + b
    model = _DummyModel()

app = Flask(__name__)

@app.route("/plot/sp500", methods=["GET"])
def plot_sp500_route():
    """Return log_sp500_close vs time as an inline image."""
    try:
        df = _load_df()
        fig = plot_log_sp500_close(df)
        return _fig_to_img_html(fig, title="log_sp500_close vs Time")
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route("/plot/vix", methods=["GET"])
def plot_vix_route():
    """Return vix_close vs time as an inline image."""
    try:
        df = _load_df()
        fig = plot_vix_close(df)
        return _fig_to_img_html(fig, title="vix_close vs Time")
    except Exception as e:
        return jsonify({"error": str(e)}), 500

def _run(): 
    app.run(port=5005, debug=False, use_reloader=False)

threading.Thread(target=_run, daemon=True).start()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5005/ (Press CTRL+C to quit)


## 7. Testing the Flask API from Notebook

### TODO: Modify examples with your actual features

In [30]:
import requests

print("SP500 plot:", requests.get("http://127.0.0.1:5005/plot/sp500").status_code)
print("VIX plot:", requests.get("http://127.0.0.1:5005/plot/vix").status_code)

sp500_html = requests.get("http://127.0.0.1:5005/plot/sp500").text
vix_html = requests.get("http://127.0.0.1:5005/plot/vix").text
with open("plot_sp500.html", "w") as f: f.write(sp500_html)
with open("plot_vix.html", "w") as f: f.write(vix_html)
print("Saved plot_sp500.html and plot_vix.html")

127.0.0.1 - - [28/Aug/2025 14:37:49] "GET /plot/sp500 HTTP/1.1" 200 -


Saved Log S&P 500 over Time.png
SP500 plot: 200


127.0.0.1 - - [28/Aug/2025 14:37:50] "GET /plot/vix HTTP/1.1" 200 -


Saved VIX over Time.png
VIX plot: 200


127.0.0.1 - - [28/Aug/2025 14:37:50] "GET /plot/sp500 HTTP/1.1" 200 -


Saved Log S&P 500 over Time.png


127.0.0.1 - - [28/Aug/2025 14:37:50] "GET /plot/vix HTTP/1.1" 200 -


Saved VIX over Time.png
Saved plot_sp500.html and plot_vix.html


## 8. Optional Streamlit / Dash Dashboard

### Add dashboard in a separate file (`app_streamlit.py` or `app_dash.py`)

## 9. Handoff Best Practices

- Ensure README.md is complete and clear
- Provide `requirements.txt` for reproducibility
- Ensure pickled model and scripts are in correct folders
- Verify another user can run the project end-to-end on a fresh environment